# 📊 Ablation Study: XAI Methods for RL-based Inventory Management
---

**Agents**: DQN (Double DQN, Per-Product Q-Network) vs A2C_mod (Actor-Critic)  
**XAI Methods**: RDX (Reward Decomposition), MSX (Minimal Sufficient Explanation), SHAP  
**Environment**: 220 products, 14 discrete actions, reward = `1 - z - overstock - q - quan`

### Experiment Grid
| Dimension | Values |
|---|---|
| Agent | DQN, A2C_mod |
| Scenario | EASY, MEDIUM, HARD |
| XAI Config | RDX_only, SHAP_only, Combined |
| λ (MSX threshold) | 0.5, 1.0, 1.5, 2.0 |

### Metrics
- **OCS** (Objective Coverage Score): fraction of objectives with |ΔQ^k| > θ_Q
- **FCS** (Feature Coverage Score): fraction of features with |SHAP| > θ_φ
- **CAS** (Cross-domain Alignment Score): Jaccard similarity between top SHAP features and top RDX objectives
- **Stability**: % MSX set change when λ varies

## Step 1: Setup & Khởi tạo Agent (Restore Checkpoints)

In [ ]:
import os, sys, warnings, time
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa   # DQN checkpoint uses tfa.layers.GroupNormalization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product as iterproduct

np.set_printoptions(edgeitems=10, linewidth=10000, precision=6, suppress=True)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print(f"TensorFlow: {tf.__version__}")
print(f"NumPy: {np.__version__}")

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================
NUM_PRODUCTS    = 220
NUM_FEATURES_PP = 3          # per product: [x, sales, q]
NUM_FEATURES    = NUM_PRODUCTS * NUM_FEATURES_PP  # 660
NUM_ACTIONS     = 14
ACTION_SPACE    = np.array([0, 0.005, 0.01, 0.0125, 0.015, 0.0175,
                            0.02, 0.03, 0.04, 0.08, 0.12, 0.2, 0.5, 1],
                           dtype=np.float32)
WASTE_RATE      = 0.025
ZERO_INVENTORY  = 1e-5
GAMMA           = 0.99

# Architecture sizes
DQN_HIDDEN  = 128
A2C_HIDDEN  = 32
DROPOUT     = 0.1

# Paths
DATA_DIR        = 'data220'
TEST_FILE       = os.path.join(DATA_DIR, 'test.tfrecords')
CAP_FILE        = os.path.join(DATA_DIR, 'capacity.tfrecords')
STOCK_FILE      = os.path.join(DATA_DIR, 'stock.tfrecords')
DQN_CKPT_DIR    = 'checkpoints_dqn_comparison3'
A2C_CKPT_DIR    = 'checkpoints'

# Reward component identifiers
OBJECTIVES = ['stockout', 'overstock', 'waste', 'quantile']
FEATURES   = ['inventory', 'sales', 'waste_feat']  # 3 input features

print("Configuration ✓")

### 1.1 Model Architecture Declarations

In [ ]:
# ============================================================
# A2C_mod Model Classes — from training1.py
# ============================================================
# Key: Critic uses tf.keras.layers.GroupNormalization(groups=1)
#      Actor has 4 Dense layers → softmax
#      Both use hidden_size=32, dropout=0.1

class Dense(tf.Module):
    def __init__(self, input_dim, output_size, activation=None, stddev=1.0):
        super(Dense, self).__init__()
        self.w = tf.Variable(
            tf.random.truncated_normal([input_dim, output_size], stddev=stddev), name='w')
        self.b = tf.Variable(tf.zeros([output_size]), name='b')
        self.activation = activation

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        if self.activation:
            y = self.activation(y)
        return y


class Actor(tf.Module):
    """Policy network: [P, 3] → [P, 14] softmax probabilities."""
    def __init__(self, num_features, num_actions, hidden_size,
                 activation=tf.nn.relu, dropout_prob=0.1):
        super(Actor, self).__init__()
        self.layer1 = Dense(num_features, hidden_size, activation=None)
        self.layer2 = Dense(hidden_size, hidden_size, activation=None)
        self.layer3 = Dense(hidden_size, hidden_size, activation=None)
        self.layer4 = Dense(hidden_size, num_actions, activation=None)
        self.activation = activation
        self.dropout_prob = dropout_prob

    def __call__(self, state):
        x = self.activation(self.layer1(state))
        x = tf.nn.dropout(x, self.dropout_prob)
        x = self.activation(self.layer2(x))
        x = tf.nn.dropout(x, self.dropout_prob)
        x = self.activation(self.layer3(x))
        x = tf.nn.dropout(x, self.dropout_prob)
        x = self.layer4(x)
        return tf.nn.softmax(x)


class Critic(tf.Module):
    """Value network: [P, 3] → [P] scalar values. Uses GroupNorm."""
    def __init__(self, num_features, hidden_size,
                 activation=tf.nn.relu, dropout_prob=0.1):
        super(Critic, self).__init__()
        self.layer1 = Dense(num_features, hidden_size, activation=None)
        self.layer2 = Dense(hidden_size, 1, activation=None)
        self.activation = activation
        self.dropout_prob = dropout_prob
        self.group_norm = tf.keras.layers.GroupNormalization(groups=1)

    def __call__(self, state):
        x = self.layer1(state)
        x = self.group_norm(x)
        x = self.activation(x)
        x = tf.nn.dropout(x, self.dropout_prob)
        x = self.layer2(x)
        return tf.squeeze(x, axis=-1, name='factor_squeeze')

print("A2C_mod classes (Dense, Actor, Critic) defined ✓")

In [ ]:
# ============================================================
# DQN Model Class — from dqn_a2c_comparison.ipynb
# Per-Product Q-Network: [B, 660] → reshape → [B*220, 3] → MLP → [B, 220, 14]
# ============================================================

class MultiProductQNetwork(tf.keras.Model):
    """
    Per-Product Q-Network. Each product processed independently.
    Input:  [B, 660]  (flattened: [x_0..x_P, sales_0..sales_P, q_0..q_P])
    Output: [B, 220, 14]
    """
    def __init__(self, num_features, num_products, num_actions,
                 hidden_size, dropout_prob=0.1, use_group_norm=True, name=None):
        super().__init__(name=name)
        self.num_products      = num_products
        self.num_actions       = num_actions
        self.features_per_prod = num_features // num_products  # 3

        self.dense1 = tf.keras.layers.Dense(hidden_size, activation=None, name="dense1")
        self.dense2 = tf.keras.layers.Dense(hidden_size, activation=None, name="dense2")
        self.dense3 = tf.keras.layers.Dense(hidden_size, activation=None, name="dense3")
        self.out    = tf.keras.layers.Dense(num_actions,  activation=None, name="output")

        self._use_gn = use_group_norm
        if use_group_norm:
            self.gn1 = tfa.layers.GroupNormalization(groups=1, name="gn1")
            self.gn2 = tfa.layers.GroupNormalization(groups=1, name="gn2")
            self.gn3 = tfa.layers.GroupNormalization(groups=1, name="gn3")

        self.drop1 = tf.keras.layers.Dropout(dropout_prob)
        self.drop2 = tf.keras.layers.Dropout(dropout_prob)
        self.drop3 = tf.keras.layers.Dropout(dropout_prob)

    def call(self, state, training=False):
        B = tf.shape(state)[0]
        P, F = self.num_products, self.features_per_prod
        # [B, 660] → [B, 3, 220] → [B, 220, 3]
        s3d = tf.transpose(tf.reshape(state, [B, F, P]), [0, 2, 1])
        x = tf.reshape(s3d, [B * P, F])

        x = self.dense1(x)
        if self._use_gn: x = self.gn1(x, training=training)
        x = tf.nn.relu(x); x = self.drop1(x, training=training)

        x = self.dense2(x)
        if self._use_gn: x = self.gn2(x, training=training)
        x = tf.nn.relu(x); x = self.drop2(x, training=training)

        x = self.dense3(x)
        if self._use_gn: x = self.gn3(x, training=training)
        x = tf.nn.relu(x); x = self.drop3(x, training=training)

        return tf.reshape(self.out(x), [B, P, self.num_actions])

print("DQN class (MultiProductQNetwork) defined ✓")

### 1.2 Load Checkpoints

In [ ]:
# ============================================================
# load_trained_agents(): Restore both agents from disk
# ============================================================

def load_trained_agents():
    """Load DQN and A2C_mod agents from checkpoints."""

    # ── A2C_mod ──────────────────────────────────────────────
    actor  = Actor(NUM_FEATURES_PP, NUM_ACTIONS, A2C_HIDDEN,
                   activation=tf.nn.relu, dropout_prob=DROPOUT)
    critic = Critic(NUM_FEATURES_PP, A2C_HIDDEN,
                    activation=tf.nn.relu, dropout_prob=DROPOUT)
    # Build with dummy pass
    _d = tf.zeros([1, NUM_FEATURES_PP])
    _ = actor(_d); _ = critic(_d)

    a2c_ckpt = tf.train.Checkpoint(
        critic_optimizer=tf.optimizers.Adam(0.0005),
        actor_optimizer=tf.optimizers.Adam(0.0001),
        critic=critic, actor=actor, step=tf.Variable(0))
    a2c_ckpt.restore(tf.train.latest_checkpoint(A2C_CKPT_DIR)).expect_partial()
    print(f"✅ A2C_mod restored: {tf.train.latest_checkpoint(A2C_CKPT_DIR)}")

    # ── DQN ──────────────────────────────────────────────────
    q_net = MultiProductQNetwork(
        NUM_FEATURES, NUM_PRODUCTS, NUM_ACTIONS,
        DQN_HIDDEN, DROPOUT, use_group_norm=True, name="q_network")
    t_net = MultiProductQNetwork(
        NUM_FEATURES, NUM_PRODUCTS, NUM_ACTIONS,
        DQN_HIDDEN, DROPOUT, use_group_norm=True, name="target_network")
    _d = tf.zeros([1, NUM_FEATURES], dtype=tf.float32)
    _ = q_net(_d, training=False); _ = t_net(_d, training=False)

    dqn_ckpt = tf.train.Checkpoint(
        optimizer=tf.optimizers.Adam(0.001),
        q_network=q_net, target_network=t_net,
        step=tf.Variable(0, dtype=tf.int64))
    dqn_ckpt.restore(tf.train.latest_checkpoint(DQN_CKPT_DIR)).expect_partial()
    print(f"✅ DQN restored: {tf.train.latest_checkpoint(DQN_CKPT_DIR)}")

    return {'actor': actor, 'critic': critic, 'q_network': q_net}

agents = load_trained_agents()

### 1.3 Load Test Data

In [ ]:
# ============================================================
# Load TFRecord test data
# ============================================================
def _parse(serialized, key, n):
    return tf.io.parse_single_example(
        serialized, {key: tf.io.FixedLenFeature([n], tf.float32)})[key]

capacity = next(iter(
    tf.data.TFRecordDataset(CAP_FILE).map(lambda s: _parse(s, 'capacity', NUM_PRODUCTS))
)).numpy()

x_init = next(iter(
    tf.data.TFRecordDataset(STOCK_FILE).map(lambda s: _parse(s, 'stock', NUM_PRODUCTS))
)).numpy()

all_sales = []
for rec in tf.data.TFRecordDataset(TEST_FILE).map(lambda s: _parse(s, 'sales', NUM_PRODUCTS)):
    all_sales.append(rec.numpy())
all_sales = np.array(all_sales, dtype=np.float32) / capacity[None, :]
T_MAX = len(all_sales)
print(f"✅ Test data: {T_MAX} timesteps × {NUM_PRODUCTS} products")

## Step 2: XAI Module — RDX, MSX, SHAP

### 2.1 RDX (Reward Decomposition eXplanation)

Phân rã Q-value/Advantage thành 4 thành phần dựa trên cấu trúc reward:
$$r = \underbrace{1}_{base} - \underbrace{z}_{stockout} - \underbrace{overstock}_{overstock} - \underbrace{q}_{waste} - \underbrace{quan}_{quantile}$$

**ΔQ^k**: Chênh lệch Q-value component k giữa action tốt nhất và action thay thế.

In [ ]:
# ============================================================
# RDX: Reward Decomposition
# ============================================================

def compute_reward_components(x_vec):
    """
    Tính 4 reward sub-components cho một trạng thái inventory x.

    Args:
        x_vec: np.ndarray [P] — inventory levels

    Returns:
        dict[str, np.ndarray[P]]: 4 components (positive = penalty magnitude)
    """
    z    = (x_vec < ZERO_INVENTORY).astype(np.float32)
    q    = WASTE_RATE * x_vec
    quan = float(np.quantile(x_vec, 0.95) - np.quantile(x_vec, 0.05))
    return {
        'stockout':  z,                                            # [P]
        'overstock': np.zeros(NUM_PRODUCTS, dtype=np.float32),     # placeholder, filled per action
        'waste':     q,                                            # [P]
        'quantile':  np.full(NUM_PRODUCTS, quan, np.float32),      # [P]
    }


def rdx_dqn(q_network, state_flat, x_vec):
    """
    RDX for DQN: Decompose Q-value difference into 4 objective contributions.

    For each product i:
      a* = argmax_a Q(s, a)_i
      a' = second-best action for product i
      ΔQ_total = Q(s, a*) - Q(s, a')
      ΔQ^k ≈ (penalty_k(a*) - penalty_k(a')) weighted by Q-value proportion

    Returns:
        delta_q: dict[str, np.ndarray[P]] — ΔQ^k per objective per product
        best_actions: np.ndarray[P] — indices of best actions
    """
    q_vals = q_network(state_flat[None, :], training=False)[0].numpy()  # [P, A]
    best_a  = np.argmax(q_vals, axis=1)    # [P]

    # Second-best action
    q_masked = q_vals.copy()
    q_masked[np.arange(NUM_PRODUCTS), best_a] = -np.inf
    second_a = np.argmax(q_masked, axis=1)  # [P]

    u_best   = ACTION_SPACE[best_a]
    u_second = ACTION_SPACE[second_a]

    # Compute penalties for best action
    os_best   = np.maximum(0, x_vec + u_best - 1)
    os_second = np.maximum(0, x_vec + u_second - 1)

    comps = compute_reward_components(x_vec)

    # ΔQ^k = difference in penalty between actions (positive = best action reduces penalty)
    delta_q = {
        'stockout':  comps['stockout'] - comps['stockout'],  # same x → same z
        'overstock': -(os_best - os_second),                 # negative sign: less overstock = better
        'waste':     comps['waste'] - comps['waste'],        # same x → same waste
        'quantile':  comps['quantile'] - comps['quantile'],  # same x → same quantile
    }

    # For stockout/waste/quantile the action doesn't change these immediately,
    # but through next-state value. Approximate via Q-value residual.
    q_best   = q_vals[np.arange(NUM_PRODUCTS), best_a]
    q_second = q_vals[np.arange(NUM_PRODUCTS), second_a]
    q_diff   = q_best - q_second  # [P]
    explained = delta_q['overstock']
    residual  = q_diff - explained

    # Distribute residual proportionally among state-dependent objectives
    penalty_magnitudes = np.stack([
        comps['stockout'], np.zeros(NUM_PRODUCTS), comps['waste'], comps['quantile']
    ])  # [4, P]
    total_pen = penalty_magnitudes.sum(axis=0, keepdims=True) + 1e-8
    weights = penalty_magnitudes / total_pen  # [4, P]

    for i, obj in enumerate(OBJECTIVES):
        if obj != 'overstock':
            delta_q[obj] = weights[i] * residual

    return delta_q, best_a


def rdx_a2c(actor, critic, state_pp, x_vec):
    """
    RDX for A2C: Decompose Advantage into 4 objective contributions.

    Advantage A(s,a) = Q(s,a) - V(s) ≈ r + γV(s') - V(s)
    Decompose r into 4 components, so A^k(s,a) ≈ r^k (immediate sub-reward)

    Returns:
        delta_q: dict[str, np.ndarray[P]]
        best_actions: np.ndarray[P]
    """
    probs    = actor(state_pp).numpy()     # [P, A]
    best_a   = np.argmax(probs, axis=1)    # [P]
    u_best   = ACTION_SPACE[best_a]

    comps  = compute_reward_components(x_vec)
    os_val = np.maximum(0, x_vec + u_best - 1)

    # Sub-reward components (sign: positive = good, negative = penalty)
    delta_q = {
        'stockout':  -comps['stockout'],
        'overstock': -os_val,
        'waste':     -comps['waste'],
        'quantile':  -comps['quantile'],
    }
    return delta_q, best_a

print("RDX module defined ✓")

### 2.2 MSX (Minimal Sufficient Explanation)

Thuật toán tìm tập hợp mục tiêu nhỏ nhất sao cho:
$$\sum_{k \in MSX} |\Delta Q^k| \geq \lambda \times Q_{threshold}$$

trong đó $Q_{threshold} = \sum_k |\Delta Q^k|$.

In [ ]:
# ============================================================
# MSX: Minimal Sufficient Explanation
# ============================================================

def compute_msx(delta_q, lam=1.0):
    """
    Find the Minimal Sufficient eXplanation set.

    Args:
        delta_q: dict[str, np.ndarray[P]] — ΔQ per objective per product
        lam: float — threshold multiplier

    Returns:
        msx_sets: list[set] — MSX set per product
        msx_sizes: np.ndarray[P] — size of MSX per product
    """
    # Stack: [4, P]
    dq_matrix = np.stack([np.abs(delta_q[obj]) for obj in OBJECTIVES])  # [4, P]
    total_dq  = dq_matrix.sum(axis=0)  # [P]
    threshold = lam * total_dq         # [P]

    msx_sets  = []
    msx_sizes = np.zeros(NUM_PRODUCTS, dtype=int)

    for p in range(NUM_PRODUCTS):
        obj_importance = [(dq_matrix[k, p], OBJECTIVES[k]) for k in range(4)]
        obj_importance.sort(key=lambda t: -t[0])  # descending by |ΔQ|

        cumsum = 0.0
        msx = set()
        for val, name in obj_importance:
            msx.add(name)
            cumsum += val
            if cumsum >= threshold[p]:
                break
        msx_sets.append(msx)
        msx_sizes[p] = len(msx)

    return msx_sets, msx_sizes


def msx_stability(delta_q, lambda_values):
    """
    Measure MSX stability across different λ values.
    Returns: float — fraction of products whose MSX changed between consecutive λ.
    """
    prev_sets = None
    changes   = []

    for lam in lambda_values:
        curr_sets, _ = compute_msx(delta_q, lam)
        if prev_sets is not None:
            n_changed = sum(1 for a, b in zip(prev_sets, curr_sets) if a != b)
            changes.append(n_changed / NUM_PRODUCTS)
        prev_sets = curr_sets

    return np.mean(changes) if changes else 0.0  # avg change rate

print("MSX module defined ✓")

### 2.3 SHAP Module

Wrapper sử dụng `shap.GradientExplainer`:
- **DQN**: SHAP trên Q-values — Input `[B, 660]`, output Q cho action được chọn
- **A2C**: SHAP trên Action Probabilities — Input `[B*P, 3]`, output softmax

In [ ]:
# ============================================================
# SHAP Module
# ============================================================
try:
    import shap
    SHAP_AVAILABLE = True
    print(f"SHAP version: {shap.__version__} ✓")
except ImportError:
    SHAP_AVAILABLE = False
    print("⚠️ SHAP not installed — SHAP_only and Combined configs will use fallback")


def shap_dqn(q_network, background_states, eval_states):
    """
    SHAP for DQN: feature importance on Q-values.

    Args:
        q_network: MultiProductQNetwork
        background_states: np.ndarray [N_bg, 660] — background dataset
        eval_states: np.ndarray [N_eval, 660] — states to explain

    Returns:
        shap_vals: np.ndarray [N_eval, 660] — SHAP values
        shap_per_feature: np.ndarray [N_eval, 3] — aggregated per feature type
            (mean abs SHAP across products for each of: inventory, sales, waste)
    """
    if not SHAP_AVAILABLE:
        # Fallback: gradient-based importance
        return _gradient_importance_dqn(q_network, eval_states)

    # Wrap model for SHAP: return mean Q across products for best action
    @tf.function
    def model_fn(x):
        q_all = q_network(x, training=False)          # [B, P, A]
        q_max = tf.reduce_max(q_all, axis=-1)          # [B, P]
        return tf.reduce_mean(q_max, axis=-1, keepdims=True)  # [B, 1]

    bg = tf.constant(background_states[:50], dtype=tf.float32)
    explainer = shap.GradientExplainer(model_fn, bg)
    sv = explainer.shap_values(tf.constant(eval_states, dtype=tf.float32))

    if isinstance(sv, list):
        sv = sv[0]
    sv = np.array(sv).reshape(len(eval_states), -1)  # [N, 660]

    # Aggregate per feature type:
    # Layout: [x_0..x_219, sales_0..sales_219, q_0..q_219]
    shap_pf = np.stack([
        np.mean(np.abs(sv[:, :220]), axis=1),       # inventory
        np.mean(np.abs(sv[:, 220:440]), axis=1),     # sales
        np.mean(np.abs(sv[:, 440:]), axis=1),         # waste
    ], axis=1)  # [N, 3]

    return sv, shap_pf


def shap_a2c(actor, background_states_pp, eval_states_pp):
    """
    SHAP for A2C Actor: feature importance on action probabilities.

    Args:
        actor: Actor model
        background_states_pp: np.ndarray [N_bg, 3] — per-product states
        eval_states_pp: np.ndarray [N_eval, 3]

    Returns:
        shap_vals: np.ndarray [N_eval, 3]
        shap_per_feature: np.ndarray [N_eval, 3]
    """
    if not SHAP_AVAILABLE:
        return _gradient_importance_a2c(actor, eval_states_pp)

    # Wrap: Actor is tf.Module, wrap in a tf.function for SHAP
    @tf.function
    def model_fn(x):
        probs = actor(x)  # [B, 14]
        return tf.reduce_max(probs, axis=-1, keepdims=True)  # [B, 1]

    bg = tf.constant(background_states_pp[:100], dtype=tf.float32)
    explainer = shap.GradientExplainer(model_fn, bg)
    sv = explainer.shap_values(tf.constant(eval_states_pp, dtype=tf.float32))

    if isinstance(sv, list):
        sv = sv[0]
    sv = np.array(sv)  # [N, 3]
    return sv, np.abs(sv)


def _gradient_importance_dqn(q_network, eval_states):
    """Fallback: gradient-based feature importance for DQN."""
    x = tf.constant(eval_states, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        q_all = q_network(x, training=False)
        q_max = tf.reduce_max(q_all, axis=-1)
        out   = tf.reduce_mean(q_max, axis=-1)
    grads = tape.gradient(out, x).numpy()  # [N, 660]
    shap_pf = np.stack([
        np.mean(np.abs(grads[:, :220]), axis=1),
        np.mean(np.abs(grads[:, 220:440]), axis=1),
        np.mean(np.abs(grads[:, 440:]), axis=1),
    ], axis=1)
    return grads, shap_pf


def _gradient_importance_a2c(actor, eval_states_pp):
    """Fallback: gradient-based feature importance for A2C."""
    x = tf.Variable(eval_states_pp, dtype=tf.float32)
    with tf.GradientTape() as tape:
        probs = actor(x)
        out   = tf.reduce_mean(tf.reduce_max(probs, axis=-1))
    grads = tape.gradient(out, x).numpy()
    return grads, np.abs(grads)

print("SHAP module defined ✓")

## Step 3: XAI Evaluation Metrics

| Metric | Formula | Description |
|--------|---------|-------------|
| **OCS** | `Σ(|ΔQ^k| > θ_Q) / 4` | Objective Coverage Score |
| **FCS** | `Σ(|SHAP_f| > θ_φ) / 3` | Feature Coverage Score |
| **CAS** | `Jaccard(top_features, mapped_objectives)` | Cross-domain Alignment |
| **Stability** | `avg % MSX change across λ` | MSX robustness |

In [ ]:
# ============================================================
# XAI EVALUATION METRICS
# ============================================================

def ocs(delta_q, theta_q=0.01):
    """
    Objective Coverage Score: fraction of objectives with |ΔQ^k| > θ_Q.
    Returns: float — mean across products.
    """
    scores = np.zeros(NUM_PRODUCTS)
    for p in range(NUM_PRODUCTS):
        n_active = sum(1 for obj in OBJECTIVES if abs(delta_q[obj][p]) > theta_q)
        scores[p] = n_active / len(OBJECTIVES)
    return float(np.mean(scores))


def fcs(shap_per_feature, theta_phi=0.001):
    """
    Feature Coverage Score: fraction of features with mean|SHAP| > θ_φ.
    shap_per_feature: [N_states, 3]
    Returns: float — mean across states.
    """
    mean_shap = np.mean(np.abs(shap_per_feature), axis=0)  # [3]
    return float(np.mean(mean_shap > theta_phi))


def cas(delta_q, shap_per_feature, theta_q=0.01, theta_phi=0.001):
    """
    Cross-domain Alignment Score using Jaccard Similarity.

    ┌─────────────────────────────────────────────────────────────────────┐
    │  FEATURE → OBJECTIVE MAPPING (for CAS computation)                │
    │                                                                     │
    │  This mapping encodes the CAUSAL RELATIONSHIP between input         │
    │  features and reward sub-components:                                │
    │                                                                     │
    │  feature 'inventory' (x) → objectives 'stockout', 'overstock'      │
    │    ■ x < zero_inv → z=1 (stockout penalty)                         │
    │    ■ x + u > 1    → overstock penalty                              │
    │    ■ Inventory is the PRIMARY driver of both these penalties.       │
    │                                                                     │
    │  feature 'sales'         → objective 'stockout'                     │
    │    ■ High sales relative to x → stockout. Sales forecasting        │
    │      accuracy directly impacts stockout risk.                       │
    │                                                                     │
    │  feature 'waste_feat' (q)→ objective 'waste'                        │
    │    ■ q = waste_rate × x: waste feature is the DIRECT input to      │
    │      the waste penalty term.                                        │
    │                                                                     │
    │  No feature maps directly to 'quantile' (it's a global statistic  │
    │  across all products, not a per-product feature).                   │
    └─────────────────────────────────────────────────────────────────────┘
    """
    FEATURE_TO_OBJECTIVE = {
        'inventory':  {'stockout', 'overstock'},
        'sales':      {'stockout'},
        'waste_feat': {'waste'},
    }

    # Top features: those with |SHAP| > θ_φ
    mean_shap = np.mean(np.abs(shap_per_feature), axis=0)  # [3]
    top_features = set()
    for i, feat in enumerate(FEATURES):
        if mean_shap[i] > theta_phi:
            top_features.add(feat)

    # Map top features → expected objectives
    expected_objectives = set()
    for feat in top_features:
        expected_objectives.update(FEATURE_TO_OBJECTIVE.get(feat, set()))

    # Top objectives: those with |ΔQ^k| > θ_Q (mean across products)
    detected_objectives = set()
    for obj in OBJECTIVES:
        if np.mean(np.abs(delta_q[obj])) > theta_q:
            detected_objectives.add(obj)

    # Jaccard similarity
    intersection = expected_objectives & detected_objectives
    union        = expected_objectives | detected_objectives
    return float(len(intersection) / max(len(union), 1))


print("Metrics (OCS, FCS, CAS, Stability) defined ✓")

## Step 4: Main Ablation Loop

Duyệt qua lưới thí nghiệm:
- **Agents**: DQN, A2C_mod
- **Scenarios**: EASY (low variance sales), MEDIUM (default), HARD (high variance, high waste)
- **Configs**: RDX_only, SHAP_only, Combined
- **λ values**: 0.5, 1.0, 1.5, 2.0

Mỗi cấu hình: lấy 496 states từ test data, chạy XAI tương ứng.

In [ ]:
# ============================================================
# SCENARIO DEFINITIONS
# Modify environment parameters to create difficulty levels
# ============================================================

SCENARIOS = {
    'EASY':   {'sales_scale': 0.5, 'waste_rate': 0.010},
    'MEDIUM': {'sales_scale': 1.0, 'waste_rate': 0.025},
    'HARD':   {'sales_scale': 1.5, 'waste_rate': 0.050},
}

XAI_CONFIGS   = ['RDX_only', 'SHAP_only', 'Combined']
LAMBDA_VALUES = [0.5, 1.0, 1.5, 2.0]
AGENT_NAMES   = ['DQN', 'A2C_mod']
N_STATES       = 496  # number of test states to evaluate

print("Experiment grid:")
total_runs = len(AGENT_NAMES) * len(SCENARIOS) * len(XAI_CONFIGS) * len(LAMBDA_VALUES)
print(f"  {len(AGENT_NAMES)} agents × {len(SCENARIOS)} scenarios × "
      f"{len(XAI_CONFIGS)} configs × {len(LAMBDA_VALUES)} λ = {total_runs} runs")

In [ ]:
# ============================================================
# COLLECT STATES FOR EACH SCENARIO
# ============================================================

def collect_states(scenario_params, n_states=N_STATES):
    """
    Run environment forward to collect n_states from test data.
    Applies scenario-specific sales scaling and waste rate.

    Returns:
        states_flat: np.ndarray [n_states, 660] — for DQN
        states_pp:   np.ndarray [n_states, P, 3] — for A2C (per-product)
        x_vecs:      np.ndarray [n_states, P]    — raw inventory vectors
    """
    scale = scenario_params['sales_scale']
    wr    = scenario_params['waste_rate']

    x = x_init.copy()
    states_flat = []
    states_pp   = []
    x_vecs      = []

    for t in range(min(n_states, T_MAX)):
        sales = all_sales[t] * scale
        q     = wr * x

        # Per-product state [P, 3]
        s_pp = np.stack([x, sales, q], axis=1).astype(np.float32)
        # Flat state [660] = [x_0..x_P, sales_0..sales_P, q_0..q_P]
        s_flat = np.concatenate([x, sales, q]).astype(np.float32)

        states_pp.append(s_pp)
        states_flat.append(s_flat)
        x_vecs.append(x.copy())

        # Step environment forward using A2C actor (default policy)
        probs = agents['actor'](s_pp).numpy()
        a_idx = np.argmax(probs, axis=1)
        u     = ACTION_SPACE[a_idx]
        x_u   = np.minimum(1, x + u)
        x     = np.maximum(0, x_u - sales)

    return (np.array(states_flat, dtype=np.float32),
            np.array(states_pp, dtype=np.float32),
            np.array(x_vecs, dtype=np.float32))

# Pre-collect states for each scenario
scenario_states = {}
for scn_name, scn_params in SCENARIOS.items():
    scenario_states[scn_name] = collect_states(scn_params)
    print(f"  Scenario {scn_name}: {scenario_states[scn_name][0].shape[0]} states collected")

print("\n✅ All scenario states collected")

In [ ]:
# ============================================================
# MAIN ABLATION LOOP
# ============================================================

results = []
t_start = time.time()

for agent_name in AGENT_NAMES:
    for scn_name in SCENARIOS:
        states_flat, states_pp, x_vecs = scenario_states[scn_name]
        n = states_flat.shape[0]

        # ── Pre-compute RDX for this (agent, scenario) ──────────
        all_delta_q = {obj: np.zeros(NUM_PRODUCTS) for obj in OBJECTIVES}
        all_best_a  = np.zeros(NUM_PRODUCTS, dtype=int)

        # Aggregate RDX across sampled states (use every 10th for speed)
        rdx_indices = np.arange(0, n, max(1, n // 50))
        for idx in rdx_indices:
            if agent_name == 'DQN':
                dq, ba = rdx_dqn(agents['q_network'],
                                  states_flat[idx], x_vecs[idx])
            else:
                dq, ba = rdx_a2c(agents['actor'], agents['critic'],
                                  states_pp[idx], x_vecs[idx])
            for obj in OBJECTIVES:
                all_delta_q[obj] += dq[obj]
            all_best_a = ba  # keep last for reference

        # Average
        for obj in OBJECTIVES:
            all_delta_q[obj] /= len(rdx_indices)

        # ── Pre-compute SHAP (agent, scenario) — only if needed ──
        shap_pf = None
        bg_idx = np.random.choice(n, min(50, n), replace=False)

        for xai_cfg in XAI_CONFIGS:
            run_rdx  = xai_cfg in ('RDX_only', 'Combined')
            run_shap = xai_cfg in ('SHAP_only', 'Combined')

            # Compute SHAP if needed and not yet computed
            if run_shap and shap_pf is None:
                eval_idx = np.random.choice(n, min(100, n), replace=False)
                if agent_name == 'DQN':
                    _, shap_pf = shap_dqn(agents['q_network'],
                                           states_flat[bg_idx], states_flat[eval_idx])
                else:
                    # Flatten per-product states for SHAP
                    bg_pp   = states_pp[bg_idx].reshape(-1, 3)
                    eval_pp = states_pp[eval_idx].reshape(-1, 3)
                    _, shap_pf = shap_a2c(agents['actor'], bg_pp, eval_pp)
                    # Average back to [N_eval, 3]
                    shap_pf = shap_pf.reshape(len(eval_idx), NUM_PRODUCTS, 3).mean(axis=1)

            for lam in LAMBDA_VALUES:
                row = {
                    'agent':    agent_name,
                    'scenario': scn_name,
                    'xai_config': xai_cfg,
                    'lambda':   lam,
                }

                # OCS
                if run_rdx:
                    row['OCS'] = ocs(all_delta_q)
                else:
                    row['OCS'] = np.nan

                # FCS
                if run_shap and shap_pf is not None:
                    row['FCS'] = fcs(shap_pf)
                else:
                    row['FCS'] = np.nan

                # CAS (only for Combined)
                if xai_cfg == 'Combined' and shap_pf is not None:
                    row['CAS'] = cas(all_delta_q, shap_pf)
                else:
                    row['CAS'] = np.nan

                # Stability
                if run_rdx:
                    row['Stability'] = 1.0 - msx_stability(all_delta_q, LAMBDA_VALUES)
                else:
                    row['Stability'] = np.nan

                # MSX size
                if run_rdx:
                    _, msx_sizes = compute_msx(all_delta_q, lam)
                    row['MSX_size_mean'] = float(msx_sizes.mean())
                else:
                    row['MSX_size_mean'] = np.nan

                results.append(row)

        # Reset shap cache for next scenario
        shap_pf = None

    print(f"  Agent {agent_name} done ({time.time()-t_start:.1f}s)")

df = pd.DataFrame(results)
df.to_csv('ablation_results.csv', index=False)
print(f"\n✅ Ablation complete: {len(df)} rows saved to ablation_results.csv")
print(f"   Total time: {time.time()-t_start:.1f}s")
df.head(10)

In [ ]:
# ============================================================
# RESULTS OVERVIEW
# ============================================================
print("=" * 70)
print("ABLATION RESULTS SUMMARY")
print("=" * 70)

# Show pivot tables
for metric in ['OCS', 'FCS', 'CAS', 'Stability', 'MSX_size_mean']:
    valid = df.dropna(subset=[metric])
    if len(valid) == 0:
        continue
    print(f"\n{'─'*50}")
    print(f"  {metric}")
    print(f"{'─'*50}")
    pivot = valid.pivot_table(values=metric,
                              index=['agent', 'scenario'],
                              columns='xai_config',
                              aggfunc='mean')
    print(pivot.round(3).to_string())

## Step 5: Visualization & Statistical Testing

### 5.1 Line Chart: Stability vs λ

In [ ]:
# ============================================================
# PLOT 1: Stability vs λ — DQN vs A2C
# ============================================================
fig, ax = plt.subplots(figsize=(10, 6))

for agent_name in AGENT_NAMES:
    subset = df[(df['agent'] == agent_name) &
                (df['xai_config'].isin(['RDX_only', 'Combined'])) &
                (df['Stability'].notna())]
    if subset.empty:
        continue
    grouped = subset.groupby('lambda')['Stability'].mean()
    ax.plot(grouped.index, grouped.values,
            marker='o', linewidth=2.5, markersize=8,
            label=agent_name)

ax.set_xlabel('λ (MSX Threshold Multiplier)', fontsize=13)
ax.set_ylabel('Stability (1 - avg % MSX change)', fontsize=13)
ax.set_title('MSX Stability vs λ: DQN vs A2C_mod', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)

plt.tight_layout()
plt.savefig('ablation_stability_vs_lambda.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: ablation_stability_vs_lambda.png")

### 5.2 Heatmap: CAS (Agent × Scenario) — Combined Config

In [ ]:
# ============================================================
# PLOT 2: CAS Heatmap — Agent × Scenario (Combined config)
# ============================================================
cas_data = df[(df['xai_config'] == 'Combined') & (df['CAS'].notna())]

if not cas_data.empty:
    pivot_cas = cas_data.pivot_table(values='CAS',
                                      index='agent',
                                      columns='scenario',
                                      aggfunc='mean')
    # Reorder columns
    col_order = [c for c in ['EASY', 'MEDIUM', 'HARD'] if c in pivot_cas.columns]
    pivot_cas = pivot_cas[col_order]

    fig, ax = plt.subplots(figsize=(8, 4))
    sns.heatmap(pivot_cas, annot=True, fmt='.3f', cmap='YlOrRd',
                vmin=0, vmax=1, linewidths=1.5, ax=ax,
                annot_kws={'fontsize': 14, 'fontweight': 'bold'})
    ax.set_title('Cross-domain Alignment Score (CAS)\nCombined XAI Config',
                 fontsize=14, fontweight='bold')
    ax.set_ylabel('Agent', fontsize=12)
    ax.set_xlabel('Scenario', fontsize=12)

    plt.tight_layout()
    plt.savefig('ablation_cas_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Saved: ablation_cas_heatmap.png")
else:
    print("⚠️ No CAS data available for Combined config")

### 5.3 Stacked Area Chart: Dominance Ratio of Reward Components

In [ ]:
# ============================================================
# PLOT 3: Stacked Area — Reward Component Dominance over States
# Uses MEDIUM scenario, both agents
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

colors = ['#E53935', '#FF9800', '#7B1FA2', '#1565C0']

for ax_idx, agent_name in enumerate(AGENT_NAMES):
    ax = axes[ax_idx]
    s_flat, s_pp, x_vs = scenario_states['MEDIUM']
    n = min(100, s_flat.shape[0])  # limit for clarity

    comp_matrix = np.zeros((n, 4))  # [n_states, 4 objectives]

    for t in range(n):
        if agent_name == 'DQN':
            dq, _ = rdx_dqn(agents['q_network'], s_flat[t], x_vs[t])
        else:
            dq, _ = rdx_a2c(agents['actor'], agents['critic'], s_pp[t], x_vs[t])

        for k, obj in enumerate(OBJECTIVES):
            comp_matrix[t, k] = np.mean(np.abs(dq[obj]))

    # Normalize to proportions
    row_sums = comp_matrix.sum(axis=1, keepdims=True) + 1e-8
    comp_pct = comp_matrix / row_sums  # [n, 4]

    x_axis = np.arange(n)
    ax.stackplot(x_axis, comp_pct.T, labels=OBJECTIVES if ax_idx == 0 else None,
                 colors=colors, alpha=0.8)
    ax.set_title(f'{agent_name}', fontsize=13, fontweight='bold')
    ax.set_xlabel('State Index')
    if ax_idx == 0:
        ax.set_ylabel('Dominance Ratio')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.2)

axes[0].legend(loc='upper left', fontsize=9, ncol=2)
plt.suptitle('Reward Component Dominance Ratio Across States (MEDIUM)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('ablation_dominance_ratio.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: ablation_dominance_ratio.png")

### 5.4 Additional: OCS Comparison and MSX Size Distribution

In [ ]:
# ============================================================
# PLOT 4: OCS Comparison across configs
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# OCS by agent and config
ocs_data = df[df['OCS'].notna()]
if not ocs_data.empty:
    pivot_ocs = ocs_data.pivot_table(values='OCS',
                                      index=['agent'],
                                      columns=['scenario'],
                                      aggfunc='mean')
    col_order = [c for c in ['EASY', 'MEDIUM', 'HARD'] if c in pivot_ocs.columns]
    pivot_ocs[col_order].plot(kind='bar', ax=axes[0], rot=0)
    axes[0].set_title('Objective Coverage Score (OCS)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('OCS')
    axes[0].legend(title='Scenario')
    axes[0].set_ylim(0, 1.1)

# MSX size by lambda
msx_data = df[df['MSX_size_mean'].notna()]
if not msx_data.empty:
    for agent_name in AGENT_NAMES:
        subset = msx_data[msx_data['agent'] == agent_name]
        grouped = subset.groupby('lambda')['MSX_size_mean'].mean()
        axes[1].plot(grouped.index, grouped.values,
                     marker='s', linewidth=2, label=agent_name)
    axes[1].set_xlabel('λ')
    axes[1].set_ylabel('Mean MSX Size')
    axes[1].set_title('MSX Set Size vs λ', fontsize=12, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ablation_ocs_msx.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: ablation_ocs_msx.png")

## Step 6: Sensitivity Analysis

### Mục tiêu
1. **Sinh dữ liệu test** cho 3 kịch bản EASY / MEDIUM / HARD bằng cách scale
   `x` (inventory), `sales`, và `waste_rate` theo tỷ lệ quy định.
2. **Giả lập thay đổi trọng số RDX**: Duyệt qua từng component ($w_s, w_h, w_w, w_o$),
   nhân RDX của nó với $\lambda \in \{0.5, 1.0, 1.5, 2.0\}$ trong khi giữ nguyên các RDX khác.
3. **Trực quan hóa**: Line chart Mean RDX theo λ + Stacked Bar chart MSX inclusion frequency.

### Scenario Data Scaling
| Parameter | EASY | MEDIUM | HARD |
|-----------|------|--------|------|
| `x` (inventory) scale | 30% | 60% | 90% |
| `sales` scale | 20% | 50% | 80% |
| `waste_rate` | 1% | 5% | 15% |

### 6.1 Sinh dữ liệu test cho các kịch bản

In [ ]:
# ============================================================
# SENSITIVITY SCENARIO DATA GENERATION
# ============================================================
# Scale factors theo yêu cầu:
#   EASY:   x=30%, sales=20%, waste_rate=1%
#   MEDIUM: x=60%, sales=50%, waste_rate=5%
#   HARD:   x=90%, sales=80%, waste_rate=15%

SENSITIVITY_SCENARIOS = {
    'EASY':   {'x_scale': 0.30, 'sales_scale': 0.20, 'waste_rate': 0.01},
    'MEDIUM': {'x_scale': 0.60, 'sales_scale': 0.50, 'waste_rate': 0.05},
    'HARD':   {'x_scale': 0.90, 'sales_scale': 0.80, 'waste_rate': 0.15},
}

def generate_scenario_data(scenario_params, n_states=N_STATES):
    """
    Sinh tập test data cho từng kịch bản Sensitivity Analysis.

    Khác với collect_states() ở Step 4:
      - x_init được SCALE theo x_scale (mô phỏng mức tồn kho ban đầu khác nhau)
      - sales được SCALE theo sales_scale (mô phỏng nhu cầu khác nhau)
      - waste_rate thay đổi theo kịch bản (không dùng WASTE_RATE toàn cục)

    Args:
        scenario_params: dict với 'x_scale', 'sales_scale', 'waste_rate'
        n_states: số lượng states cần sinh

    Returns:
        states_flat: np.ndarray [n_states, 660] — cho DQN
        states_pp:   np.ndarray [n_states, P, 3] — cho A2C (per-product)
        x_vecs:      np.ndarray [n_states, P]    — inventory vectors thô
    """
    x_sc  = scenario_params['x_scale']
    s_sc  = scenario_params['sales_scale']
    wr    = scenario_params['waste_rate']

    # Scale initial inventory
    x = (x_init * x_sc).astype(np.float32)

    states_flat, states_pp, x_vecs = [], [], []

    for t in range(min(n_states, T_MAX)):
        # Scale sales
        sales = (all_sales[t] * s_sc).astype(np.float32)
        # Waste with scenario-specific rate
        q = (wr * x).astype(np.float32)

        # Build state representations
        s_pp   = np.stack([x, sales, q], axis=1).astype(np.float32)      # [P, 3]
        s_flat = np.concatenate([x, sales, q]).astype(np.float32)  # [660]

        states_pp.append(s_pp)
        states_flat.append(s_flat)
        x_vecs.append(x.copy())

        # Step environment forward (A2C policy)
        probs = agents['actor'](s_pp).numpy()
        a_idx = np.argmax(probs, axis=1)
        u     = ACTION_SPACE[a_idx]
        x_u   = np.minimum(1, x + u)
        x     = np.maximum(0, x_u - sales).astype(np.float32)

    return (np.array(states_flat, np.float32),
            np.array(states_pp, np.float32),
            np.array(x_vecs, np.float32))

# Generate data for each sensitivity scenario
sens_data = {}
for scn_name, scn_params in SENSITIVITY_SCENARIOS.items():
    sens_data[scn_name] = generate_scenario_data(scn_params)
    n = sens_data[scn_name][0].shape[0]
    print(f"  {scn_name}: {n} states | "
          f"x_scale={scn_params['x_scale']:.0%}, "
          f"sales_scale={scn_params['sales_scale']:.0%}, "
          f"waste_rate={scn_params['waste_rate']:.0%}")

print("\n✅ Sensitivity scenario data generated")

### 6.2 Giả lập thay đổi trọng số RDX

Ý tưởng: Với mỗi component $w_k \in \{w_s, w_h, w_w, w_o\}$:
- Tính RDX gốc: $\Delta Q^k$ cho tất cả 4 objectives
- Nhân $\Delta Q^k$ của component đang xét với $\lambda$, giữ nguyên các component khác
- Quan sát: Mean RDX thay đổi thế nào? MSX set thay đổi thế nào?

Mapping ký hiệu:
- $w_s$ → `stockout` (z)
- $w_h$ → `overstock`
- $w_w$ → `waste` (q)
- $w_o$ → `quantile` (quan)

In [ ]:
# ============================================================
# PERTURB RDX WEIGHTS & COLLECT SENSITIVITY METRICS
# ============================================================

# Component weights notation: w_s=stockout, w_h=overstock, w_w=waste, w_o=quantile
WEIGHT_LABELS = {
    'stockout':  '$w_s$ (Stockout)',
    'overstock': '$w_h$ (Overstock)',
    'waste':     '$w_w$ (Waste)',
    'quantile':  '$w_o$ (Quantile)',
}

def perturb_rdx_weights(delta_q_original, target_component, lam):
    """
    Giả lập thay đổi trọng số của MỘT component RDX.

    Nhân ΔQ^k của target_component với λ, giữ nguyên các component khác.
    → Mô phỏng việc "tăng/giảm tầm quan trọng" của một mục tiêu.

    Args:
        delta_q_original: dict[str, np.ndarray[P]] — RDX gốc
        target_component: str — component cần thay đổi ('stockout', 'overstock', ...)
        lam: float — hệ số scale

    Returns:
        delta_q_perturbed: dict[str, np.ndarray[P]] — RDX đã điều chỉnh
    """
    perturbed = {}
    for obj in OBJECTIVES:
        if obj == target_component:
            perturbed[obj] = delta_q_original[obj] * lam   # Scale component này
        else:
            perturbed[obj] = delta_q_original[obj].copy()   # Giữ nguyên
    return perturbed


# ── Run sensitivity analysis ───────────────────────────────────
SENS_LAMBDAS = [0.5, 1.0, 1.5, 2.0]

sensitivity_results = []

for agent_name in AGENT_NAMES:
    for scn_name in SENSITIVITY_SCENARIOS:
        s_flat, s_pp, x_vs = sens_data[scn_name]
        n = s_flat.shape[0]

        # Compute baseline RDX (average over sampled states)
        rdx_base = {obj: np.zeros(NUM_PRODUCTS) for obj in OBJECTIVES}
        sample_idx = np.arange(0, n, max(1, n // 50))

        for idx in sample_idx:
            if agent_name == 'DQN':
                dq, _ = rdx_dqn(agents['q_network'], s_flat[idx], x_vs[idx])
            else:
                dq, _ = rdx_a2c(agents['actor'], agents['critic'],
                                 s_pp[idx], x_vs[idx])
            for obj in OBJECTIVES:
                rdx_base[obj] += dq[obj]
        for obj in OBJECTIVES:
            rdx_base[obj] /= len(sample_idx)

        # Perturb each component independently
        for target_comp in OBJECTIVES:
            for lam in SENS_LAMBDAS:
                dq_perturbed = perturb_rdx_weights(rdx_base, target_comp, lam)

                # Mean absolute RDX for perturbed component
                mean_rdx = float(np.mean(np.abs(dq_perturbed[target_comp])))

                # MSX inclusion: how often does target_comp appear in MSX?
                msx_sets, _ = compute_msx(dq_perturbed, lam=1.0)  # MSX threshold fixed at 1.0
                inclusion_rate = sum(1 for ms in msx_sets if target_comp in ms) / NUM_PRODUCTS

                sensitivity_results.append({
                    'agent':     agent_name,
                    'scenario':  scn_name,
                    'component': target_comp,
                    'w_label':   WEIGHT_LABELS[target_comp],
                    'lambda':    lam,
                    'mean_rdx':       mean_rdx,
                    'msx_inclusion':  inclusion_rate,
                })

    print(f"  Sensitivity done for {agent_name}")

df_sens = pd.DataFrame(sensitivity_results)
df_sens.to_csv('sensitivity_results.csv', index=False)
print(f"\n✅ Sensitivity analysis: {len(df_sens)} rows saved to sensitivity_results.csv")
df_sens.head(8)

### 6.3 Visualization: Sensitivity Analysis

In [ ]:
# ============================================================
# PLOT SA-1: Line Chart — Mean |ΔQ^k| vs λ (per component)
# ============================================================
# Trục X: λ (hệ số scale của component đang xét)
# Trục Y: Mean |ΔQ^k| (giá trị RDX trung bình sau khi nhân λ)
# Mỗi subplot = 1 agent, 4 đường cho 4 components

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=False)

comp_colors = {
    'stockout':  '#E53935',
    'overstock': '#FF9800',
    'waste':     '#7B1FA2',
    'quantile':  '#1565C0',
}

for ax_idx, agent_name in enumerate(AGENT_NAMES):
    ax = axes[ax_idx]

    for comp in OBJECTIVES:
        subset = df_sens[(df_sens['agent'] == agent_name) &
                         (df_sens['component'] == comp)]
        if subset.empty:
            continue

        # Average across scenarios for cleaner visualization
        grouped = subset.groupby('lambda')['mean_rdx'].mean()

        ax.plot(grouped.index, grouped.values,
                marker='o', linewidth=2.5, markersize=8,
                color=comp_colors[comp],
                label=WEIGHT_LABELS[comp])

    ax.set_xlabel('λ (Scale Factor)', fontsize=12)
    ax.set_ylabel('Mean |ΔQ^k|', fontsize=12)
    ax.set_title(f'{agent_name}: Mean RDX vs λ', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10, loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_xticks(SENS_LAMBDAS)

plt.suptitle('Sensitivity Analysis: How RDX Magnitude Responds to Weight Scaling',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('sensitivity_mean_rdx_vs_lambda.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: sensitivity_mean_rdx_vs_lambda.png")

In [ ]:
# ============================================================
# PLOT SA-2: 100% Stacked Bar — MSX Inclusion Frequency
# ============================================================
# Khi λ của component k thay đổi, tần suất k lọt vào MSX thay đổi thế nào?
# Mỗi thanh bar = 1 giá trị λ, chia thành 4 phần (4 components)
# Chiều cao mỗi phần = inclusion rate của component đó trong MSX

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

bar_colors = ['#E53935', '#FF9800', '#7B1FA2', '#1565C0']

for ax_idx, agent_name in enumerate(AGENT_NAMES):
    ax = axes[ax_idx]

    # For each lambda, get inclusion rates when THAT component is the perturbed one
    # We want to show: when we scale w_k by λ, how often does k appear in MSX?
    inclusion_matrix = np.zeros((len(OBJECTIVES), len(SENS_LAMBDAS)))

    for k, comp in enumerate(OBJECTIVES):
        for j, lam in enumerate(SENS_LAMBDAS):
            subset = df_sens[(df_sens['agent'] == agent_name) &
                             (df_sens['component'] == comp) &
                             (df_sens['lambda'] == lam)]
            if not subset.empty:
                inclusion_matrix[k, j] = subset['msx_inclusion'].mean()

    # Normalize columns to 100% for stacked bar
    col_sums = inclusion_matrix.sum(axis=0, keepdims=True)
    col_sums = np.where(col_sums == 0, 1, col_sums)  # avoid div by 0
    inc_pct = inclusion_matrix / col_sums  # [4, len(λ)]

    x_pos = np.arange(len(SENS_LAMBDAS))
    bar_width = 0.6
    bottom = np.zeros(len(SENS_LAMBDAS))

    for k, comp in enumerate(OBJECTIVES):
        ax.bar(x_pos, inc_pct[k], bar_width, bottom=bottom,
               color=bar_colors[k], label=WEIGHT_LABELS[comp],
               alpha=0.85, edgecolor='white', linewidth=0.5)
        # Annotate percentage
        for j in range(len(SENS_LAMBDAS)):
            if inc_pct[k, j] > 0.05:  # only label if visible
                ax.text(x_pos[j], bottom[j] + inc_pct[k, j] / 2,
                        f'{inc_pct[k, j]:.0%}',
                        ha='center', va='center', fontsize=8, fontweight='bold')
        bottom += inc_pct[k]

    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'λ={v}' for v in SENS_LAMBDAS])
    ax.set_xlabel('λ (Scale Factor of Perturbed Component)', fontsize=11)
    ax.set_ylabel('MSX Inclusion Proportion', fontsize=11)
    ax.set_title(f'{agent_name}', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.05)
    ax.legend(fontsize=9, loc='upper right', ncol=2)

plt.suptitle('MSX Inclusion Frequency: How Component Importance Changes with Weight Scaling',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('sensitivity_msx_inclusion_stacked.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: sensitivity_msx_inclusion_stacked.png")

In [ ]:
# ============================================================
# SENSITIVITY SUMMARY TABLE
# ============================================================
print("=" * 70)
print("SENSITIVITY ANALYSIS SUMMARY")
print("=" * 70)

# Pivot: Mean RDX by (agent, component) vs lambda
pivot_rdx = df_sens.pivot_table(
    values='mean_rdx',
    index=['agent', 'component'],
    columns='lambda',
    aggfunc='mean'
)
print("\n📊 Mean |ΔQ^k| across λ values:")
print(pivot_rdx.round(4).to_string())

# Pivot: MSX inclusion by (agent, component) vs lambda
pivot_inc = df_sens.pivot_table(
    values='msx_inclusion',
    index=['agent', 'component'],
    columns='lambda',
    aggfunc='mean'
)
print("\n📊 MSX Inclusion Rate across λ values:")
print(pivot_inc.round(3).to_string())

# Key insight: which component is most sensitive?
for agent_name in AGENT_NAMES:
    agent_data = df_sens[df_sens['agent'] == agent_name]
    rdx_range = agent_data.groupby('component')['mean_rdx'].agg(['min', 'max'])
    rdx_range['sensitivity'] = rdx_range['max'] - rdx_range['min']
    most_sensitive = rdx_range['sensitivity'].idxmax()
    print(f"\n🔍 {agent_name}: Most sensitive component = {most_sensitive} "
          f"(range = {rdx_range.loc[most_sensitive, 'sensitivity']:.4f})")

## Summary & Interpretation

### How to Read Results

**OCS (Objective Coverage Score)**:
- High OCS → agent's decisions are driven by multiple reward objectives
- Low OCS → agent focuses narrowly on 1-2 objectives

**FCS (Feature Coverage Score)**:
- High FCS → model uses all input features (inventory, sales, waste) in decisions
- Low FCS → model relies on subset of features

**CAS (Cross-domain Alignment)**:
- High CAS → SHAP feature importances AGREE with RDX objective importances
- Low CAS → disconnect between what features the model uses and what objectives it optimizes

**Stability**:
- High Stability → MSX explanation is robust across threshold changes
- Low Stability → explanations are fragile, sensitive to hyperparameters

### Sensitivity Analysis Interpretation
- **Line chart (SA-1)**: Nếu đường dốc → component đó có ảnh hưởng mạnh lên giải thích
- **Stacked bar (SA-2)**: Nếu tỷ lệ phần trăm thay đổi nhiều → MSX nhạy cảm với trọng số

**For Reviewers**: The CAS metric is the key contribution — it bridges
the gap between feature-level (SHAP) and objective-level (RDX) explanations,
validating that the model's internal feature usage aligns with the reward
structure of the environment.